# 逻辑回归模型

## 1. 模型算法

### 1.1 模型简介
逻辑回归模型假设`label`服从伯努利分布，通过极大化似然函数的方法，运用梯度下降法进行参数的求解，从而解决二分类问题。

### 1.2 模型假设

- `label`服从伯努利分布
假设正样本的概率为$p$，负样本的概率则为$1-p$，那么整个逻辑回归模型可以描述为如下，
$$h_\theta(\theta;x)=p$$
- 概率假设
逻辑回归模型假设正样本的概率$p$为如下，
$$p=\frac{1}{1+e^{-\theta^Tx}}$$
相当于对于特征做了线性加权，之后对于加权值做了一个非线性变换(**sigmoid**函数)

### 1.3 模型流程
#### 1.3.1 损失函数
由训练样本得到最大似然函数$L$，如下，
$$L=\prod_{i=1}^m p_i^{y_i}.(1-p_i)^{1-y_i}$$
其对数似然函数为，
$$\ln L=\sum_{i=1}^m y_i\ln{p_i}+(1-y_i)\ln{(1-p_i)} $$
最大化似然函数对应最小化负的似然函数，那么对似然函数取负数即可得到逻辑回归模型的损失函数，如下，
$$\rm{Loss}=- \sum_{i=1}^m y_i\ln{p_i}+(1-y_i)\ln{(1-p_i)}$$
其中上式也称之为**交叉熵**,由熵的定义($H(p)=-\sum_{i=1}^m p_i\log{p_i}$)演化得到。

#### 1.3.2 模型求解
模型求解的过程即通过最小化损失函数，从而求得参数$\theta$的过程，即
$$\theta^*=\underset{\theta}{\arg \min}\ \rm{Loss}$$
通常会在损失函数项中加入正则项($L_1,L_2$)，具体形式分别如下，
$$\underset{\theta,b}{\min}||\theta||_1+C \cdot \rm{Loss}$$
$$\underset{\theta,b}{\min}\frac{1}{2}\theta^T\theta+C \cdot \rm{Loss}$$
其中$C$表示正则项的控制力度，$C$越小表示强的正则强度

求解方法如下表（参考`sklearn`中逻辑回归算法的求解方式），

| case| solver|
| :-- | :-- |
| 不添加正则项| 梯度下降| 
| $L_1$正则项| `CD`or`saga` | 
| $L_2$正则项 | `lbfgs` `newton-cg` `sag` `saga`|
| 大数据集 | `sag` `saga`|


#### 1.3.3 example  

下面以**梯度下降**解法为例，进行求解，假设数据为，
\begin{array}{ccc|c}
x_1 & {x_2} & {x_3} & {label} \\
\hline
8 & 5& 1 & 0\\
1 & 4 & 3& 1 \\
3 & 2& 2& 0 \\
3 & 3& 0& 0 \\
3 & 10 & 1& 1 \\
3 & 9 & 8& 0 
\end{array}

则损失函数为
$$Loss(\theta_j)=-\sum_{i=1}^{6}y_i\ln p_i+(1-y_i)\ln (1-p_i)$$
其中$p_i=\frac{1}{1+e^{-\theta^Tx}}=\frac{1}{1+e^{\theta_0+\theta_1x_1+\theta_2x_2+\theta_3x_3}}$
损失函数$\rm{Loss}(\theta_j)$关于$\theta_j$ 的偏导数为如下，
\begin{align*}
\frac{\partial\ \rm{Loss}(\theta_j)}{\partial\ \theta_j} & = -\sum_{i=1}^6y_i\cdot\frac{1}{p_i}\cdot \frac{\partial\ p_i}{\partial\ \theta_j}+(1-y_i)\cdot\frac{1}{1-p_i}\cdot (-1) \cdot \frac{\partial\ p_i}{\partial\ \theta_j} \\
 & = -\sum_{i=1}^{6} y_i\cdot \frac{1}{p_i}\cdot p_i\cdot(1-p_i)\cdot x_j^{(i)} + (1-y_i)\cdot\frac{1}{1-p_i}\cdot (-1) \cdot p_i\cdot(1-p_i)\cdot x_j^{(i)}\\ 
 & =   -\sum_{i=1}^{6} y_i\cdot(1-p_i)\cdot x_j^{(i)} + (1-y_i)\cdot (-1) \cdot p_i\cdot x_j^{(i)}\\
 & = -\sum_{i=1}^{6}(p_i-y_i)\cdot x_j^{(i)} \\ 
\end{align*}
其中$\frac{\partial\ p_i}{\partial\ \theta_j}=p_i\cdot(1-p_i)\cdot x_j^{(i)}$.

求解的过程如下，
1. 随机初始化参数${\theta}$, $\theta_0=1,\theta_1=1,\theta_2=1, \theta_3=1$
2. 依次更新$\theta_0,\theta_1,\theta_2,\theta_3$，更新方式如下，
$$\theta_j := \theta_j-\alpha \cdot \left. \frac{\partial\ \rm{Loss}(\theta_j)}{\partial\ \theta_j} \right|_{\theta_j},j=0,1,2,3$$
其中$\alpha$为预先设定的学习率。      

3. 不断重复上述步骤2，直到参数值收敛或者达到预定训练次数。  

那么最终的模型为,
$$h(\theta,x) = \frac{1}{1+e^{\theta_0+\theta_1x_1+\theta_2x_2+\theta_3x_3}}$$


### 1.3.4 逻辑回归模型疑问

- 损失函数为什么使用最大似然估计而不是最小二乘法？    
从最优化的角度来看，如果选用最小二乘法，那么损失函数可以写为
$$\rm Loss=\sum_{i=1}^m(p_i-y_i)^2$$
那么$\rm Loss$是关于参数$\theta$的非凸函数，难以求得最优解。
从模型假设来看，逻辑回归模型假设`label`服从伯努利分布，往往数据`label`服从高斯分布（正态分布）时使用平方损失。
- 逻辑回归模型在训练时，如果有很多特征高度相关，那么会有什么影响？    
在最终收敛的情况下，即使存在多个线性相关的特征，最终的模型精度一样。整体模型训练的速度会变慢。
- 如果有特征存在量纲不一致的情况，比如某个特征取值范围在0-1之间，另外一个特征取值在1-100000之间，最终有什么影响？  
在保证训练次数的前提下，最终的模型会收敛，模型精度一致。不过模型整体训练的速度会变慢。
